In [1]:
import os
import package_combiner as PC
import pandas as pd
import simpy
import random
from datetime import datetime, timedelta

current_dir = os.getcwd()
folder_path = os.path.join(current_dir,'Packages_TP_Test')
output_file = 'combined_packages.csv' 

combined_df = PC.combine_and_convert_packages(folder_path, output_file)




/home/dataminer/workspace/package_combiner.py:20: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/home/dataminer/workspace/package_combiner.py:20: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


comined CSV file saved as combined_packages.csv
Timestamp Corrections Applied


In [2]:
combined_df_2=pd.read_csv("combined_packages.csv")

filtered_df = combined_df_2.loc[combined_df_2['sortation_center'] == 3858]
filtered_df = filtered_df[filtered_df['scdre.scac'] == 'TLMD']

###
features_to_keep = ['pkg_received_utc_ts','scdre.package_tracking_number','scdre.route_sort_c', 'scdre.sortation_center_package_sort_partition']

df_demand = filtered_df.reset_index()

df_demand = df_demand[features_to_keep]

df_demand.rename(columns={'pkg_received_utc_ts':'pkg_received_utc_ts'}, inplace = True)

df_demand.rename(columns={'scdre.package_tracking_number':'package_tracking_number'}, inplace = True)

df_demand.rename(columns={'scdre.route_sort_c':'route_sort_c'}, inplace = True)

df_demand.rename(columns={'scdre.sortation_center_package_sort_partition':'sortation_center_package_sort_partition'}, inplace = True)

                                          
def seconds_since_start(timestamp):
    ref_time = timestamp.replace(hour=17,minute=0,second=0,microsecond=0)

    if timestamp <ref_time:
        ref_time -= timedelta(days=1)

    delta = timestamp-ref_time

    return int(delta.total_seconds())


In [3]:
import simpy
import pandas as pd
import random

class G:
# Constants (adjust as needed)
    PRODUCTIVITY_RATE = 1  #Assumed productivity rate for simplicity

    UNLOADING_RATE = 5  #pallets per hour
    INDUCT_STAGE_RATE = 5 #pallets per hour
    INDUCTION_RATE = 5 #packages per hour
    SPLITTER_RATE = 5 #packages per hour
    TLMD_BUFFER_SORT_RATE = 5 #packages per hour
    NATIONAL_CARRIER_SORT_RATE = 5 #packages per hour
    TLMD_PALLET_TRANSFER_RATE = 5 #pallets per hour
    TLMD_FEEDER_RATE = 5 #pallets per hour
    TLMD_INDUCTION_STAGE_RATE = 5 #pallets per hour
    TLMD_INDUCTION_RATE = 5 # packages per hour
    TLMD_PRESORT_RATE = 5 #packages per hour
    TLMD_SORT_RATE = 5 #packages per hour
    TLMD_CART_STAGE_RATE = 5 #carts per hour
    TLMD_CART_HANDOFF_RATE = 5 #carts per hour
    NATIONAL_CARRIER_LOAD_RATE = 5 #pallets per hour

    INDUCT_PALLET_CAPACITY = 200 #pallets/loadingdock
    AVE_TLMD_PALLET_CAPACITY = 35 #packages/pallet
    AVE_CART_CAPACITY = 20 #packages/cart
    MAX_PALLETS_STAGE = 50 #pallets/partition
    STAGE_CAPACITY = 160 #carts/staging area



class Package:
    def __init__(self, env, pkg_received_utc_ts, package_tracking_number, route_sort_c, sortation_center_package_sort_partition):
        self.env = env
        self.pkg_received_utc_ts = pkg_received_utc_ts
        self.package_tracking_number = package_tracking_number
        self.route_sort_c = route_sort_c
        self.sortation_center_package_sort_partition = sortation_center_package_sort_partition
        self.current_queue = None

class Sortation_Center:
    def __init__(self, env, packages_df, day_num_tm_pit, day_num_tm_nonpit, night_num_tm_pit, night_num_tm_nonpit, num_pallet_jacks):
        self.env = env
        self.packages_df = packages_df
        self.day_num_tm_pit = day_num_tm_pit
        self.day_num_tm_nonpit = day_num_tm_nonpit
        self.night_num_tm_pit = night_num_tm_pit
        self.night_num_tm_nonpit = night_num_tm_nonpit
        self.num_pallet_jacks = num_pallet_jacks

        self.shift_start_times = [0, 13.5]  # Shifts start at 17:00 (0) and 06:30 (13)
        self.shift_end_times = [10, 23.5]  # Shifts end at 03:00 (10) and 16:30 (23.5)
        self.break_times = [(3, 3.5), (7, 7.25), (16.5, 17), (20.5, 20.75)]  # Breaks at 20:00-20:30, 00:00-00:15, 09:30-10:00, 13:30-13:45

        self.day_tm_pit = simpy.Resource(env, capacity = day_num_tm_pit)
        self.day_tm_nonpit = simpy.Resource(env, capacity = day_num_tm_nonpit)
        self.night_tm_pit = simpy.Resource(env, capacity = night_num_tm_pit)
        self.night_tm_nonpit = simpy.Resource(env, capacity = night_num_tm_nonpit)
        self.pallet_jack = simpy.Resource(env, num_pallet_jacks)

        # Initialize queues
        self.init_queues()

        # Schedule initial package arrivals
        self.schedule_arrivals()

    def init_queues(self):
        self.queues = {
            'queue_inbound_truck': simpy.Store(self.env),

            'queue_inbound_staging': simpy.Store(self.env, capacity=1000),

            'queue_induct_staging': simpy.Store(self.env, capacity=1000),

            'queue_national_carrier_split': simpy.Store(self.env, capacity=1000),

            'queue_national_carrier_sort': simpy.Store(self.env, capacity=1000),

            'queue_TLMD_buffer': simpy.Store(self.env, capacity=1000),

            'queue_Partition1_staging': simpy.Store(self.env, capacity=200),
            'queue_Partition2_staging': simpy.Store(self.env, capacity=200),
            'queue_Partition3_staging': simpy.Store(self.env, capacity=200),

            'queue_TLMD_induct_staging': simpy.Store(self.env, capacity=1000),

            
            'queue_presort_CDE': simpy.Store(self.env, capacity=1000),
            'queue_presort_FGH': simpy.Store(self.env, capacity=1000),
            'queue_presort_IJK': simpy.Store(self.env, capacity=1000),
            'queue_presort_LMN': simpy.Store(self.env, capacity=1000),

            'queue_Group_AB': simpy.Store(self.env, capacity=1000),
            'queue_Group_CDE': simpy.Store(self.env, capacity=1000),
            'queue_Group_FGH': simpy.Store(self.env, capacity=1000),
            'queue_Group_IJK': simpy.Store(self.env, capacity=1000),
            'queue_Group_LMN': simpy.Store(self.env, capacity=1000),

            'queue_Group_A_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_B_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_C_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_D_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_E_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_F_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_G_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_H_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_I_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_J_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_K_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_L_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_M_Cart': simpy.Store(self.env, capacity=600),
            'queue_Group_N_Cart': simpy.Store(self.env, capacity=600),

            'queue_cart_staging': simpy.Store(self.env, capacity=1000),

            'queue_USPS_Pallet': simpy.Store(self.env, capacity=1000),
            'queue_UPSN_4_A01_Pallet': simpy.Store(self.env, capacity=1000),
            'queue_FDEG_4_B01_Pallet': simpy.Store(self.env, capacity=1000),
            'queue_FDEG_4_C01_Pallet': simpy.Store(self.env, capacity=1000),
            'queue_FDE_4_D01_Pallet': simpy.Store(self.env, capacity=1000),

            'queue_USPS_truck': simpy.Store(self.env, capacity=1000),
            'queue_UPSN_4_A01_truck': simpy.Store(self.env, capacity=1000),
            'queue_FDEG_4_B01_truck': simpy.Store(self.env, capacity=1000),
            'queue_FDEG_4_C01_truck': simpy.Store(self.env, capacity=1000),
            'queue_FDE_4_D01_truck': simpy.Store(self.env, capacity=1000)
        }

        self.sinks = {
            'sink_tlmd_delivered': simpy.Store(self.env),
            'sink_missed_tlmd_cpt': simpy.Store(self.env),
            'sink_nc_delivered': simpy.Store(self.env),
            'sink_missed_nc_cpt': simpy.Store(self.env)
        }

###########################################################################################
    def run_shifts(self):

        shift_1_start = 0
        shift_1_end = shift_1_start + 10*60
        shift_2_start = shift_1_end + 3.5*60
        shift_2_end = shift_2_start + 10*60
        break_1_start = shift_1_start + 3*60
        break_1_end = break_1_start +.5*60
        break_2_start = break_1_end + 3.5*60
        break_2_end = break_2_start + 0.25*60
        break_3_start = shift_2_start + 3*60
        break_3_end = break_3_start + 0.5*60
        break_4_start = break_3_end + 3.5*60
        break_4_end = break_4_start + .25*60

        ######################

        shift_times = [(shift_1_start, shift_1_end, self.night_tm_pit, self.night_tm_nonpit),
                       (shift_2_start, shift_2_end, self.day_tm_pit, self.day_tm_nonpit)]
        
        break_times = [(break_1_start, break_1_end), (break_2_start, break_2_end), (break_3_start, break_3_end), (break_4_start, break_4_end)]
        
        for shift_start, shift_end, pit_resource, nonpit_resource in shift_times:
            self.env.process(self.run_shift(shift_start, shift_end,pit_resource, nonpit_resource))

        for break_start, break_end in break_times:
            self.env.process(self.take_break(break_start, break_end))

    def take_break(self, break_start, break_end):
        yield self.env.timeout(break_start)
        yield self.env.timeout(break_end-break_start)

    def run_shift(self, shift_start, shift_end, pit_resource, nonpit_resource):
        while True:
            if shift_start <= shift_end:
                yield self.env.timeout(shift_end-shift_start)

            else:
                yield self.env.timeout(1) #why is this every minute? could I do every 30 minutes?

    def schedule_arrivals(self):
        for i, row in self.packages_df.iterrows():
            package = Package(self.env, row['pkg_received_utc_ts'], row['package_tracking_number'],
                              row['route_sort_c'], row['sortation_center_package_sort_partition'])
            self.env.process(self.truck_arrival(package))

    def get_current_resources(self):
        shift_1_start = 0
        shift_1_end = shift_1_start + 10*60
        shift_2_start = shift_1_end + 3.5*60
        shift_2_end = shift_2_start + 10*60

        if shift_1_start <= self.env.now < shift_1_end:
            return self.night_tm_pit, self.night_tm_nonpit
        
        elif shift_2_start <= self.env.now < shift_2_end:
            return self.day_tm_pit, self.day_tm_nonpit
        
        else:
            return None,None

    def truck_arrival(self, package):
        yield self.env.timeout(package.pkg_received_utc_ts)
        package.current_queue = 'queue_inbound_truck'
        yield self.queues['queue_inbound_truck'].put(package)
        self.env.process(self.unload_truck(package))

    def unload_truck(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.pit_resource.request() as req1, self.pallet_jack.request() as req2:
            yield req1 & req2
            yield self.env.timeout(G.UNLOADING_RATE)  # Unloading time
            package.current_queue = 'queue_inbound_staging'
            yield self.queues['queue_inbound_staging'].put(package)
            self.env.process(self.move_to_induct_staging(package))

    def move_to_induct_staging(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.pit_resource.request() as req1, self.pallet_jack.request() as req2:
            yield req1 & req2
            yield self.env.timeout(G.INDUCT_STAGE_RATE)  # Moving time
            package.current_queue = 'queue_induct_staging'
            yield self.queues['queue_induct_staging'].put(package)
            self.env.process(self.induct_package(package))

    def induct_package(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.pit_resource.request() as req:
            yield req
            yield self.env.timeout(G.INDUCTION_RATE)  # Induction time
            package.current_queue = 'queue_national_carrier_split'
            yield self.queues['queue_national_carrier_split'].put(package)
            self.env.process(self.national_carrier_split(package))

    def national_carrier_split(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.SPLITTER_RATE)  # Splitting time
            if package.sortation_center_package_sort_partition in ['UPSN_4_A01', 'USPS', 'FDEG_4_C01', 'FDEG_4_B01', 'FDE_4_D01']:
                package.current_queue = 'queue_national_carrier_sort'
                yield self.queues['queue_national_carrier_sort'].put(package)
                self.env.process(self.national_carrier_sort(package))
            else:
                package.current_queue = 'queue_TLMD_buffer'
                yield self.queues['queue_TLMD_buffer'].put(package)
                self.env.process(self.TLMD_buffer_sort(package))

    def national_carrier_sort(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.NATIONAL_CARRIER_SORT_RATE)  # Sorting time
            if package.sortation_center_package_sort_partition == 'UPSN_4_A01':
                package.current_queue = 'queue_UPSN_4_A01_Pallet'
                yield self.queues['queue_UPSN_4_A01_Pallet'].put(package)
            elif package.sortation_center_package_sort_partition == 'USPS':
                package.current_queue = 'queue_USPS_Pallet'
                yield self.queues['queue_USPS_Pallet'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDEG_4_C01':
                package.current_queue = 'queue_FDEG_4_C01_Pallet'
                yield self.queues['queue_FDEG_4_C01_Pallet'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDEG_4_B01':
                package.current_queue = 'queue_FDEG_4_B01_Pallet'
                yield self.queues['queue_FDEG_4_B01_Pallet'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDE_4_D01':
                package.current_queue = 'queue_FDE_4_D01_Pallet'
                yield self.queues['queue_FDE_4_D01_Pallet'].put(package)
            self.env.process(self.load_carrier_truck(package))

    def load_carrier_truck(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.pit_resource.request() as req1, self.pallet_jack.request() as req2:
            yield req1 & req2
            yield self.env.timeout(G.NATIONAL_CARRIER_LOAD_RATE)  # Loading time
            if package.sortation_center_package_sort_partition == 'UPSN_4_A01':
                package.current_queue = 'queue_UPSN_4_A01_truck'
                yield self.queues['queue_UPSN_4_A01_truck'].put(package)
            elif package.sortation_center_package_sort_partition == 'USPS':
                package.current_queue = 'queue_USPS_truck'
                yield self.queues['queue_USPS_truck'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDEG_4_C01':
                package.current_queue = 'queue_FDEG_4_C01_truck'
                yield self.queues['queue_FDEG_4_C01_truck'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDEG_4_B01':
                package.current_queue = 'queue_FDEG_4_B01_truck'
                yield self.queues['queue_FDEG_4_B01_truck'].put(package)
            elif package.sortation_center_package_sort_partition == 'FDE_4_D01':
                package.current_queue = 'queue_FDE_4_D01_truck'
                yield self.queues['queue_FDE_4_D01_truck'].put(package)

    def TLMD_buffer_sort(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_BUFFER_SORT_RATE)  # Sorting time
            if package.sortation_center_package_sort_partition == 'Partition1':
                package.current_queue = 'queue_Partition1_staging'
                yield self.queues['queue_Partition1_staging'].put(package)
            elif package.sortation_center_package_sort_partition == 'Partition2':
                package.current_queue = 'queue_Partition2_staging'
                yield self.queues['queue_Partition2_staging'].put(package)
            elif package.sortation_center_package_sort_partition == 'Partition3':
                package.current_queue = 'queue_Partition3_staging'
                yield self.queues['queue_Partition3_staging'].put(package)
            self.env.process(self.TLMD_induct_stage(package))

    def TLMD_induct_stage(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_INDUCTION_STAGE_RATE)  # Induction time
            package.current_queue = 'queue_TLMD_induct_staging'
            yield self.queues['queue_TLMD_induct_staging'].put(package)
            self.env.process(self.TLMD_induct(package))

    def TLMD_induct(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_INDUCTION_RATE)  # Induction time
            if package.route_sort_c in ['A', 'B']:
                package.current_queue = 'queue_Group_AB'
                yield self.queues['queue_Group_AB'].put(package)
            elif package.route_sort_c in ['C', 'D', 'E']:
                package.current_queue = 'queue_presort_CDE'
                yield self.queues['queue_presort_CDE'].put(package)
            elif package.route_sort_c in ['F', 'G', 'H']:
                package.current_queue = 'queue_presort_FGH'
                yield self.queues['queue_presort_FGH'].put(package)
            elif package.route_sort_c in ['I', 'J', 'K']:
                package.current_queue = 'queue_presort_IJK'
                yield self.queues['queue_presort_IJK'].put(package)
            elif package.route_sort_c in ['L', 'M', 'N']:
                package.current_queue = 'queue_presort_LMN'
                yield self.queues['queue_presort_LMN'].put(package)
            self.env.process(self.TLMD_presort_groups(package))

    def TLMD_presort_groups(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_PRESORT_RATE)  # Presorting time
            if package.route_sort_c in ['C', 'D', 'E']:
                package.current_queue = 'queue_Group_CDE'
                yield self.queues['queue_Group_CDE'].put(package)
            elif package.route_sort_c in ['F', 'G', 'H']:
                package.current_queue = 'queue_Group_FGH'
                yield self.queues['queue_Group_FGH'].put(package)
            elif package.route_sort_c in ['I', 'J', 'K']:
                package.current_queue = 'queue_Group_IJK'
                yield self.queues['queue_Group_IJK'].put(package)
            elif package.route_sort_c in ['L', 'M', 'N']:
                package.current_queue = 'queue_Group_LMN'
                yield self.queues['queue_Group_LMN'].put(package)
            self.env.process(self.TLMD_sort_groups(package))


    def TLMD_sort_groups(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_SORT_RATE)  # Sorting time
            if package.route_sort_c == 'A':
                package.current_queue = 'queue_Group_A_Cart'
                yield self.queues['queue_Group_A_Cart'].put(package)
            elif package.route_sort_c == 'B':
                package.current_queue = 'queue_Group_B_Cart'
                yield self.queues['queue_Group_B_Cart'].put(package)
            elif package.route_sort_c == 'C':
                package.current_queue = 'queue_Group_C_Cart'
                yield self.queues['queue_Group_C_Cart'].put(package)
            elif package.route_sort_c == 'D':
                package.current_queue = 'queue_Group_D_Cart'
                yield self.queues['queue_Group_D_Cart'].put(package)
            elif package.route_sort_c == 'E':
                package.current_queue = 'queue_Group_E_Cart'
                yield self.queues['queue_Group_E_Cart'].put(package)
            elif package.route_sort_c == 'F':
                package.current_queue = 'queue_Group_F_Cart'
                yield self.queues['queue_Group_F_Cart'].put(package)
            elif package.route_sort_c == 'G':
                package.current_queue = 'queue_Group_G_Cart'
                yield self.queues['queue_Group_G_Cart'].put(package)
            elif package.route_sort_c == 'H':
                package.current_queue = 'queue_Group_H_Cart'
                yield self.queues['queue_Group_H_Cart'].put(package)
            elif package.route_sort_c == 'I':
                package.current_queue = 'queue_Group_I_Cart'
                yield self.queues['queue_Group_I_Cart'].put(package)
            elif package.route_sort_c == 'J':
                package.current_queue = 'queue_Group_J_Cart'
                yield self.queues['queue_Group_J_Cart'].put(package)
            elif package.route_sort_c == 'K':
                package.current_queue = 'queue_Group_K_Cart'
                yield self.queues['queue_Group_K_Cart'].put(package)
            elif package.route_sort_c == 'L':
                package.current_queue = 'queue_Group_L_Cart'
                yield self.queues['queue_Group_L_Cart'].put(package)
            elif package.route_sort_c == 'M':
                package.current_queue = 'queue_Group_M_Cart'
                yield self.queues['queue_Group_M_Cart'].put(package)
            elif package.route_sort_c == 'N':
                package.current_queue = 'queue_Group_N_Cart'
                yield self.queues['queue_Group_N_Cart'].put(package)
            self.env.process(self.cart_staging(package))

    def cart_staging(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_CART_STAGE_RATE)  # Staging time
            package.current_queue = 'queue_cart_staging'
            yield self.queues['queue_cart_staging'].put(package)
            self.env.process(self.load_carrier_truck(package))

            """
##########################################################
##########################################################
#Requires significant work
    def load_TLMD_vehicle(self, package):
        pit_resource, nonpit_resource = self.get_current_resources()
        with self.nonpit_resource.request() as req:
            yield req
            yield self.env.timeout(G.TLMD_CART_HANDOFF_RATE)  # Loading time
            if package.sortation_center_package_sort_partition in ['UPSN_4_A01', 'USPS', 'FDEG_4_C01', 'FDEG_4_B01', 'FDE_4_D01']:
                package.current_queue = f'queue_{package.sortation_center_package_sort_partition}_truck'
                yield self.queues[package.current_queue].put(package)
                # Final delivery process would go here

#############################################################
            """
            
    def run_simulation(self, until_time):
        self.env.run(until=until_time)



In [4]:
# Example usage
if __name__ == "__main__":
    # Create a sample dataframe for package arrival data

    packages_df = df_demand
    
    def adjust_custom_day(timestamp):
        if timestamp.time() >= pd.Timestamp('17:00:00').time():
            return timestamp.date()
        else:
            return (timestamp-pd.Timedelta(days=1)).date()
        
    packages_df.set_index('pkg_received_utc_ts', inplace = True)
        
    packages_df['custom_day'] = packages_df['pkg_received_utc_ts'].apply(adjust_custom_day)
    
    for day, group in packages_df.groupby('custom_day'):
        
        df_demand['pkg_received_utc_ts'] = df_demand['pkg_received_utc_ts'].apply(seconds_since_start)

        # Initialize the simulation environment
        env = simpy.Environment()

        # Initialize the TLMD system with parameters
        day_num_tm_pit = 5
        day_num_tm_nonpit = 10
        night_num_tm_pit = 10
        night_num_tm_nonpit = 20

        num_pallet_jacks = 20

        tlmd_system = Sortation_Center(env, packages_df, day_num_tm_pit, day_num_tm_nonpit, night_num_tm_pit, night_num_tm_nonpit, num_pallet_jacks)

        # Run the simulation for a specific duration (e.g., 24 hours)
        tlmd_system.run_simulation(until_time=24*60)  # Simulate 24 hours in minutes


KeyError: 'timestamp'